In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
import timm
import cv2
import dlib
from tensorflow.keras.preprocessing import image
import math

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision.models import mobilenet_v2

In [ ]:
dataset_path = '/content/drive/MyDrive/three_final/'

In [ ]:
class_labels = []

for item in os.listdir(dataset_path):
    all_classes = os.listdir(os.path.join(dataset_path, item))
    for room in all_classes:
        class_labels.append((item, os.path.join(dataset_path, item, room)))

class_labels[:5]

[('sad', '/content/drive/MyDrive/three_final/sad/sad_1592.jpg'),
 ('sad', '/content/drive/MyDrive/three_final/sad/sad_1593.jpg'),
 ('sad', '/content/drive/MyDrive/three_final/sad/sad_1594.jpg'),
 ('sad', '/content/drive/MyDrive/three_final/sad/sad_1595.jpg'),
 ('sad', '/content/drive/MyDrive/three_final/sad/sad_1596.jpg')]

In [ ]:
df = pd.DataFrame(data = class_labels, columns = ['Labels', 'image'])
print(df.head())

  Labels                                              image
0    sad  /content/drive/MyDrive/three_final/sad/sad_159...
1    sad  /content/drive/MyDrive/three_final/sad/sad_159...
2    sad  /content/drive/MyDrive/three_final/sad/sad_159...
3    sad  /content/drive/MyDrive/three_final/sad/sad_159...
4    sad  /content/drive/MyDrive/three_final/sad/sad_159...


In [ ]:
df.tail()

,Labels,image
6976,happy,/content/drive/MyDrive/three_final/happy/happy...
6977,happy,/content/drive/MyDrive/three_final/happy/happy...
6978,happy,/content/drive/MyDrive/three_final/happy/happy...
6979,happy,/content/drive/MyDrive/three_final/happy/happy...
6980,happy,/content/drive/MyDrive/three_final/happy/happy...


In [ ]:
print('total num of images in the dataset : ', len(df))

label_count = df['Labels'].value_counts()
print(label_count)

total num of images in the dataset :  6981
neutral    2410
happy      2327
sad        2244
Name: Labels, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

print(len(train_df))

5025


In [ ]:
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset

# 라벨 인코딩
label_encoder = LabelEncoder()
label_encoder.fit(df['Labels'])

train_labels = label_encoder.transform(train_df['Labels'])
val_labels = label_encoder.transform(val_df['Labels'])
test_labels = label_encoder.transform(test_df['Labels'])

# 원핫 인코딩
train_targets = torch.nn.functional.one_hot(torch.tensor(train_labels))
val_targets = torch.nn.functional.one_hot(torch.tensor(val_labels))
test_targets = torch.nn.functional.one_hot(torch.tensor(test_labels))

In [ ]:
val_labels

array([1, 2, 1, ..., 1, 0, 0])

In [ ]:
val_targets

tensor([[0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        ...,
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0]])

In [ ]:
# 이미지 전처리 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, df, targets, transform=None):
        self.df = df
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path = self.df['image'].iloc[index]
        img_label = self.targets[index]

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img, img_label

In [ ]:
im_size = 224

In [ ]:
transform = transforms.Compose([
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# 데이터셋 생성
train_dataset = ImageDataset(train_df, train_labels, transform=transform)
val_dataset = ImageDataset(val_df, val_labels, transform=transform)
test_dataset = ImageDataset(test_df, test_labels, transform=transform)

In [ ]:
# DataLoader 생성
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# 모델 정의 (MobileNetV2)
model = mobilenet_v2(pretrained=True)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = torch.nn.Linear(num_ftrs, len(df['Labels'].unique()))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 97.5MB/s]


In [ ]:
# 모델 학습
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 30
best_val_loss = float('inf')
patience = 29
early_stopping_counter = 0

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}')
    # Validation 세트를 사용하여 성능 검증 및 Early Stopping
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for val_images, val_labels in val_dataloader:
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)

            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()

    val_loss /= len(val_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}')
    # Early Stopping 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= patience:
            print('Early Stopping!')
            break

Epoch [1/30], Train Loss: 0.6792
Epoch [1/30], Validation Loss: 0.5693
Epoch [2/30], Train Loss: 0.5501
Epoch [2/30], Validation Loss: 0.5533
Epoch [3/30], Train Loss: 0.4990
Epoch [3/30], Validation Loss: 0.5699
Epoch [4/30], Train Loss: 0.4896
Epoch [4/30], Validation Loss: 0.8754
Epoch [5/30], Train Loss: 0.4606
Epoch [5/30], Validation Loss: 0.6138
Epoch [6/30], Train Loss: 0.4569
Epoch [6/30], Validation Loss: 0.5029
Epoch [7/30], Train Loss: 0.4352
Epoch [7/30], Validation Loss: 0.4895
Epoch [8/30], Train Loss: 0.3977
Epoch [8/30], Validation Loss: 1.0181
Epoch [9/30], Train Loss: 0.4867
Epoch [9/30], Validation Loss: 0.5552
Epoch [10/30], Train Loss: 0.3960
Epoch [10/30], Validation Loss: 0.6194
Epoch [11/30], Train Loss: 0.3696
Epoch [11/30], Validation Loss: 0.6741
Epoch [12/30], Train Loss: 0.3447
Epoch [12/30], Validation Loss: 0.5320
Epoch [13/30], Train Loss: 0.3163
Epoch [13/30], Validation Loss: 0.5584
Epoch [14/30], Train Loss: 0.3183
Epoch [14/30], Validation Loss: 0.5

In [ ]:
# 테스트 데이터셋에 대한 예측
model.load_state_dict(torch.load('best_model.pt'))
model.eval()

test_loss = 0.0
test_correct = 0
total_samples = 0

with torch.no_grad():
    for test_images, test_labels in test_dataloader:
        test_images = test_images.to(device)
        test_labels = test_labels.to(device)

        test_outputs = model(test_images)
        _, predicted = torch.max(test_outputs.data, 1)

        test_loss += criterion(test_outputs, test_labels).item()
        test_correct += (predicted == test_labels).sum().item()
        total_samples += test_labels.size(0)

test_accuracy = test_correct / total_samples
test_loss /= len(test_dataloader)

print('Test Accuracy:', test_accuracy)
print('Test Loss:', test_loss)

Test Accuracy: 0.7739628040057225
Test Loss: 0.5361950952898372


In [ ]:
# import cv2
# path = '/content/drive/MyDrive/three_final/'
# dataset_path = os.listdir('/content/drive/MyDrive/three_final/')

# im_size = 224

# images = []
# labels = []

# for i in dataset_path:
#   data_path = path + str(i)
#   filenames = [i for i in os.listdir(data_path)]

#   for f in filenames:
#     img = cv2.imread(data_path +'/' + f)
#     img = cv2.resize(img, (im_size, im_size))
#     images.append(img)
#     labels.append(i)

In [ ]:
images[10].shape

In [ ]:
#데이터 전처리
transform = transforms.Compose([
    transforms.ToTensor(),  # 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

In [ ]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.1, stratify=labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, stratify=train_labels, random_state=42)

In [ ]:
# 하이퍼파라미터 설정
num_classes = 3  # 분류할 표정 클래스 수
batch_size = 8
num_epochs = 100
learning_rate = 0.001
patience = 5

In [ ]:
train_images = [np.array(image) for image in train_images]
train_images = np.stack(train_images)
val_images = [np.array(image) for image in val_images]
val_images = np.stack(val_images)
test_images = [np.array(image) for image in test_images]
test_images = np.stack(test_images)

In [ ]:
# 데이터셋 생성
train_dataset = TensorDataset(torch.from_numpy(train_images), torch.tensor(train_labels))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(torch.from_numpy(val_images), torch.tensor(val_labels))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = TensorDataset(torch.from_numpy(test_images), torch.tensor(test_labels))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)